# Regression models

In this notebook we show how the tuning library can be called for any regression predictor implementing our `fit` and `evaluate` methods.
The tuning process can be differentiated based on a number of different settings, most importantly the set of parameters to be explored and the method of evaluation. The latter is by default set to train-test split for timing reasons (since GridSearch is a computationally expensive procedure). However more robust results are likely using `method='CV'`

In [ ]:
import pandas as pd
import numpy as np

import sys
sys.path.append('..')

from regressors import RidgePredictor, LassoPredictor, RandomForestPredictor
from tuning import tune, bayesian_optimization
from preprocessing import keep_intersection_of_columns

In [ ]:
# load datasets and define writhing path
aggregated_train = pd.read_csv('../data/aggregated_train.csv', dtype={"fullVisitorId": str})
aggregated_test = pd.read_csv('../data/aggregated_test.csv', dtype={"fullVisitorId": str})
write_to = '../data/tuning.txt'

In [ ]:
assert aggregated_test.shape[0] == 617242, ("ABORT, ABORT! TEST DATA DOES NOT HAVE THE RIGHT SHAPE! Make sure your"
                                            " data is created with the latest version of preprocessing.py.")

__Important:__ Drop the target variable and the visitor ID from the data.

In [ ]:
train_x, test_x = keep_intersection_of_columns(aggregated_train.reset_index(drop=True), aggregated_test.reset_index(drop=True))
train_y = aggregated_train.reset_index(drop=True)["target_sum"]
train_x = train_x.fillna(0).drop(["target_sum", "fullVisitorId"], axis=1)
test_ids = test_x["fullVisitorId"]
test_x = test_x.fillna(0).drop(["target_sum", "fullVisitorId"], axis=1)

In [ ]:
param_grid = {
    'max_depth': [3,4,5,6]
}

best_params, best_score = tune(RandomForestPredictor, train_x, train_y, param_grid, silent=False, persist=True, write_to=write_to, method = 'CV')
print("Optimal parameters achieve RMSE = {}".format(best_score))
print("Parameters: {}".format(best_params))

In [ ]:
param_grid = {
    'alpha': np.logspace(-10, -2, 15),
    'normalize': [True], # only false if the data is not normalized!!
}

best_params, best_score = tune(RidgePredictor, train_x, train_y, param_grid, silent=False, persist=True, write_to=write_to, method = 'CV')
print("Optimal parameters achieve RMSE = {}".format(best_score))

In [ ]:
param_grid = {
    'alpha': np.logspace(-10, -2, 15),
    'normalize': [True] # only false if the data is not normalized!!
}

best_params, best_score = tune(LassoPredictor, train_x, train_y, param_grid, silent=False, persist=False, write_to=write_to, method = 'CV')
print("Optimal parameters achieve RMSE = {}".format(best_score))

### Make a submission

In [ ]:
from submit import create_submission
params = {"max_depth": 3}
submission = create_submission(RandomForestPredictor, params, train_x, train_y, test_x, test_ids,
                               save=True, path="../submissions/submission.csv")
submission.head()